# Candy Crush in Python - Part 4 - Removing Matches and Filling in Blank Spaces

In this lesson we will write the code to take the horizontal and vertical matches, remove them from the board, and then fill in the blank spaces with either the tiles from above, or new tiles if there are no tiles above.

# Code Recap

So far we have:
* Our game board and function to display the grid
* A function to swap two cells beside eachother
* A function to find all matches of a particular length on the board

In [ ]:
import numpy as np
from gamegrid import GameGrid
from IPython.display import display

def show_board(board_array):
    grid = GameGrid()
    grid.data = board_array
    display(grid)
    
def swap_items(board, first, second):    
    dist = np.sum(np.abs(np.subtract(second, first)))
    
    is_beside = dist <= 1
    
    if(is_beside):
        tmp = board[first]
        board[first] = board[second]        
        board[second] = tmp 

def horiz_match_length(board, point):
    first = board[point]
    (r, c) = point
    (row_count, col_count) = board.shape
    same_tile_count = 1
    
    for i in range(c+1, col_count):
        if(board[r, i] != first):
            return same_tile_count
        else:
            same_tile_count = same_tile_count + 1
            
    return same_tile_count

def vert_match_length(board, point):
    first = board[point]
    (r,c) = point
    (row_count, col_count) = board.shape
    same_tile_count = 1    
    
    for i in range(r+1, row_count):
        if(board[i,c] != first):
            return same_tile_count
        else:
            same_tile_count = same_tile_count + 1
            
    return same_tile_count

def find_matches(board, required_matches):
    horiz_matches = []
    vert_matches = []
    
    (row_count, col_count) = board.shape
    
    for r in range(row_count):
        for c in range(col_count):
            point = (r,c)
            
            horiz_len = horiz_match_length(board, point)
            if horiz_len >= required_matches:
                horiz_matches.append((r, c, horiz_len))  
            
            vert_len = vert_match_length(board, point)
            if vert_len >= required_matches:
                vert_matches.append((r, c, vert_len))
                
    return (horiz_matches, vert_matches)

And we can use our code as follows:

In [ ]:
find_all_board = np.array([
    [2, 1, 2, 3],
    [2, 0, 0, 0],
    [2, 3, 0, 3],
    [4, 2, 0, 3]    
])

show_board(find_all_board)

(h_matches, v_matches) = find_matches(find_all_board, 3)
print("Horizontal: "+ str(h_matches))
print("Vertical: " + str(v_matches))

## Removing the Matching Tiles

In the game, whenever we get three or more of the same tile either horizontally, we want to remove the matching tiles, and the tiles above drop into the space left. We want to be able to display on screen the blank spaces for a brief time before the tiles above drop, so we will do this in two steps. 

On our game board each of the different gem stones are represented by a number `0`-`4`, we now need to add the concept of an empty space as well. We could just say that an empty space is represented by the number `5`, but there are two issues with this:

1. We might decide to add more types of tile later on, which would mean changing all the code that refers to a blank space as `5`. 
2. A blank space is fundamentally different from the other tiles, we want to make this distinction very clear, which will make the code easier to follow. 

Normally we would choose pythons 'None' type for this purpose, but our game board is an array of integers, which does not allow 'None', so instead we choose a what is called 'sentinel value' - the number `-1`.

We need to now write a function remove_matches() that takes our horizontal and vertical matches and sets all matching tiles to `-1`.

### Writing the Function

The remove function is quite simple, it loops over our horizontal matche and every vertical match and it sets the value of every cell in that match to the value `-1`. 

In [ ]:
def remove_matching_tiles(board, horiz_matches, vert_matches):
    for (r, c, length) in horiz_matches:
         for i in range(c, c+length):
                board[r, i] = -1
    
    for (r, c, length) in vert_matches:
        for i in range(r, r+length):
            board[i, c] = -1

The function uses a nested for loop (one for loop inside another). Each match consists of a tuple `(r, c, length)` - the row, column and number of cells in the match.

The outer loop `for (r, c, length) in horiz_matches:` does two things: first, loop over every horizontal match and second unpack the match tuple into the variables `r`, `c` and `length`.

The inner loop `for i in range(c, c+length):` loops over every cell in the match, our match starts at column `c` and ends at that column plus the number of cells in the match which is equal to `c+length`.

Inside the inner loop, we set the cell at `(r, i)` to -1.

The logic for the vertical matches is the same, except we switch `r` and `i`.

### Using The Function

So lets test with a new board:

In [ ]:
remove_board = np.array([
    [0, 1, 1, 1],
    [2, 3, 4, 2],
    [2, 4, 3, 1],
    [2, 3, 2, 1]
])

print(remove_board)
show_board(remove_board)

We can see from this board that we have a match in the first row starting at `(0,1)`, and a match in the first column starting at `(1,0)`, both of length `3`. So lets call our remove_matching_tiles function with these matches and display the result.

In [ ]:
h_matches = [[0,1,3]]
v_matches = [[1,0,3]]
remove_matching_tiles(remove_board, h_matches, v_matches)
print(remove_board)

When we call our function on the board we see that the matched tiles have now all been replaced with `-1`. When we display our game board now we see that the tiles are all blank.

In [ ]:
show_board(remove_board)

## Filling the Empty Spaces

Now that we have our game grid with all the matching tiles removed, we want to fill them in with new tiles. In the case of our example board above, we want to move the yellow diamond in tile at `(0, 0)` down to `(3, 0)`. We then want to fill any remaining spaces with random gems.

### Pulling Cells Down

We will start by writing a function that, given an empty cell, it will search for the first non-empty cell above it, and 'pull' the gem down into it.

Lets take a specific example: We want a function that when given the cell `(3, 0)` (the blank cell in the last row) will search upwards until it finds the yellow diamond in `(0, 0)` and pull it down. Our function must first check the cell immediately above it `(2, 0)` - which is empty - then the one above that `(1, 0)` - also empty - before finally finding our yellow diamond in cell `(0, 0)`. 

What we need is a loop that can check the cells `(2,0)`, `(1,0)`, and `(0,0)`. The 0 stays the same each time, all that changes in each iteration is the row - which should run in the order `[2, 1, 0]`. The `range(3)` function will loop in the order `[0, 1, 2]` and we can use the reversed() function to change this to `[2, 1, 0]`:

In [ ]:
r = 3
for i in reversed(range(3)):
    print(i)

We can now use this to loop though all the cells above the cell we are looking at, in this case `(3, 2)`

In [ ]:
r = 3
c = 0
for i in reversed(range(r)):
    print(str((i, c)) + " = " + str(remove_board[i, c]))

So this loop will allow us to check, the cell immediately above `(2, 0)`, then the next cell up `(1, 0)` and finally the top cell `(0,0)` - which is where we find our yellow diamond.

### Creating the Function

To create our function, we use this loop to find until we find a tile that does not have the value `-1`, and then pull that tile down, and make the tile it was in blank.

In [ ]:
def pull_down_cell(board, point):
    (r, c) = point
    for r_above in reversed(range(r)):    
        if(board[r_above, c] != -1):            
            board[r,c] = board[r_above,c]
            board[r_above,c] = -1
            break

### Using the Function

We can test this on our game grid:

In [ ]:
pull_down_board = np.array([
    [0, -1, -1, -1],
    [-1, 3, 4, 2],
    [-1, 4, 3, 1],
    [-1, 3, 2, 1]
])

show_board(pull_down_board)

In [ ]:
pull_down_cell(pull_down_board, (3, 0))
show_board(pull_down_board)

We can see that the yellow diamond has been pulled down into the bottom row.

## Filling in Blank Cells

What if we try our function on a cell with no gem stones above it e.g. `(2, 0)`?

In [ ]:
pull_down_cell(pull_down_board, (2, 0))
show_board(pull_down_board)

Nothing changes, because there are no tiles in above to pull down, in this case we just want to put a random gem in the tile. For this we will use numpys's `random.randint()` function. We need to tell `randint(...)` what range of numbers we want. In this example we want numbers from `0`-`4`, which means we should call `np.random.randint(5)`, but we might change the number of types of tile later on, so we should pass this in as a parameter instead:

In [ ]:
def pull_down_cell(board, point, number_of_types):
    (r, c) = point
    for r_above in reversed(range(r)):    
        if(board[r_above, c] != -1):            
            board[r,c] = board[r_above,c]
            board[r_above,c] = -1
            break
    # If after we have pulled down the cell is still blank
    #  insert a blank cell
    if(board[r,c] == -1):
        board[r,c] = np.random.randint(number_of_types)

And lets try it out:

In [ ]:
pull_down_board = np.array([
    [-1, -1, -1, -1],
    [-1, 3, 4, 2],
    [-1, 4, 3, 1],
    [0, 3, 2, 1]
])
pull_down_cell(pull_down_board, (2, 0), 5)
show_board(pull_down_board)

## Pulling Down all Cells

Now we can see that a random new tile has been added to the `(2, 0)`.
Finally, we want to do this for every cell in our grid so we create a function `pull_down_cells()`, that will do this on every cell. We must however do this from the last row to the first - if we went from the first to the last we would still end up with lots of blank spaces left after we have completed. So we will use the `reversed()` function again to do the loop backwards.

In [ ]:
def pull_down_cells(board, number_of_types):
    for r in reversed(range(board.shape[0])):
        for c in range(board.shape[1]):
            if(board[r,c] == -1):
                pull_down_cell(board, (r, c), number_of_types)

Lets recreate our original game grid with the empty spaces and run the `pull_down_cells(...)` function on it.

In [ ]:
pull_down_all_board = np.array([
    [0, -1, -1, -1],
    [-1, 3, 4, 2],
    [-1, 4, 3, 1],
    [-1, 3, 2, 1]
])

pull_down_cells(pull_down_all_board, 5)
show_board(pull_down_all_board)

## Summary

In this lesson we learned how to remove the matches that we found in the previous lesson, and to either pull down tiles from above into the empty spaces, or fill an empty space with a random tile if there are no tiles above.

In the next section we will look at: [Adding Interactivity](Part5-Interactivity.ipynb)